In [59]:
import numpy as np
from scipy import sparse
from time import time
import pandas as pd
import networkx as nx
import graph_tool as gt
import matplotlib.pyplot as plt
import seaborn as sns
import json
import regex as re
from pathlib import Path
from copy import deepcopy
from importlib import reload
from collections import Counter, defaultdict

In [29]:
import analysis
import files
import grouping

In [68]:
reload(grouping)

<module 'grouping' from '/home/rimov/Documents/Code/Studies/reddit-assortativity-analysis/grouping.py'>

In [3]:
%matplotlib notebook
plt.rcParams.update({'axes.titlesize': 'large'})
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [12]:
paths['SouthPark']

KeyError: 'SouthPark'

In [5]:
paths = files.get_network_paths()
paths

{'TheRedPill': PosixPath('data/chain-networks/TheRedPill.json'),
 'ketorecipes': PosixPath('data/chain-networks/ketorecipes.json'),
 'gameofthrones': PosixPath('data/chain-networks/gameofthrones.json'),
 'aggies': PosixPath('data/chain-networks/aggies.json'),
 'Rateme': PosixPath('data/chain-networks/Rateme.json'),
 'answers': PosixPath('data/chain-networks/answers.json'),
 'ABraThatFits': PosixPath('data/chain-networks/ABraThatFits.json'),
 'Warhammer40k': PosixPath('data/chain-networks/Warhammer40k.json'),
 'networking': PosixPath('data/chain-networks/networking.json'),
 'dataisbeautiful': PosixPath('data/chain-networks/dataisbeautiful.json'),
 'AsianHotties': PosixPath('data/chain-networks/AsianHotties.json'),
 'StonerPhilosophy': PosixPath('data/chain-networks/StonerPhilosophy.json'),
 'NewSkaters': PosixPath('data/chain-networks/NewSkaters.json'),
 'Scotland': PosixPath('data/chain-networks/Scotland.json'),
 'homelab': PosixPath('data/chain-networks/homelab.json'),
 'Blowjobs': Po

In [4]:
groups = files.get_groups()
groups.keys()

dict_keys(['fashion', 'market', 'media', 'sports', 'medical', 'politics', 'professional', 'sexual', 'other', 'science', 'violent', 'health_fitness', 'travel', 'philosophy', 'books_comics', 'personal', 'tech', 'news', 'vehicles', 'music', 'religion', 'video_games', 'food', 'education', 'geography', 'comedy', 'q_and_a', 'hobbies', 'animals', 'stories'])

In [26]:
groups

{'fashion': ['femalefashionadvice',
  'MakeupAddiction',
  'malegrooming',
  'ABraThatFits',
  'FancyFollicles',
  'malehairadvice',
  'braswap',
  'BeautyBoxes',
  'TF2fashionadvice',
  'AustralianMakeup',
  'caps',
  'streetwear',
  'SkincareAddiction',
  'BeardPorn',
  'piercing',
  'frugalmalefashion',
  'TeenMFA',
  'malefashionadvice',
  'malefashion',
  'MakeupAddicts',
  'curvy',
  'asianamerican',
  'AsianBeauty',
  'Hair',
  'supremeclothing',
  'burstingout',
  'tightdresses',
  'entwives',
  'MaleFashionMarket',
  'Sneakers'],
 'market': ['airsoftmarket',
  'freebies',
  'GameTrade',
  'AdoptMyVillager',
  'kickstarter',
  'smallbusiness',
  'churning',
  'SteamGameSwap',
  'freedonuts',
  'AmateurArchives',
  'CryptoCurrency',
  'vaporents',
  'StockMarket',
  'Flipping',
  'ACTrade',
  'BitcoinMarkets',
  'Foodforthought',
  'investing',
  'Random_Acts_Of_Pizza',
  'tf2trade',
  'rotmgtradingpost',
  'Ebay',
  'ecigclassifieds',
  'makeupexchange',
  'Dota2Trade',
  'Vape

In [ ]:
grouping.get_missing_added_changed()

In [45]:
sample = {g: list(np.random.choice(srs, 10, replace=False)) for g,srs in groups.items()}
sample

{'fashion': ['Sneakers',
  'BeautyBoxes',
  'malefashionadvice',
  'caps',
  'TeenMFA',
  'malefashion',
  'femalefashionadvice',
  'frugalmalefashion',
  'malegrooming',
  'BeardPorn'],
 'market': ['churning',
  'Pmsforsale',
  'ecigclassifieds',
  'airsoftmarket',
  'YGOSales',
  'CryptoCurrency',
  'freedonuts',
  'forhire',
  'vaporents',
  'dogemarket'],
 'media': ['InsideGaming',
  'italy',
  'scifi',
  'pornvids',
  'howardstern',
  'videos',
  'moviescirclejerk',
  'oddlysatisfying',
  'somethingimade',
  'SouthPark'],
 'sports': ['leafs',
  'nl_Kripparrian',
  'MLS',
  'sports',
  'Seahawks',
  'Dodgers',
  'formula1',
  'osugame',
  'FinalFantasy',
  'CFB'],
 'medical': ['diabetes',
  'pharmacy',
  'kratom',
  'depression',
  'Dentistry',
  'ems',
  'medicine',
  'doctorwho',
  'autism',
  'opiates'],
 'politics': ['CanadaPolitics',
  'berkeley',
  'ShitAmericansSay',
  'HailCorporate',
  'worldpolitics',
  'argentina',
  'MensRights',
  'canada',
  'Russian',
  'pakistan'],


In [46]:
sample['media'].append('NotASubreddit1')
sample['politics'].append('NotASubreddit2')
sample['other'].append('NotASubreddit3')

In [53]:
diff = dict()
orig = set(paths.keys())

for g,srs in sample.items():
    s = set(srs)
    
    diff[g] = list(s.difference(orig))

In [54]:
diff

{'fashion': [],
 'market': [],
 'media': ['SouthPark', 'NotASubreddit1'],
 'sports': [],
 'medical': [],
 'politics': ['NotASubreddit2', 'Russian'],
 'professional': [],
 'sexual': [],
 'other': ['NotASubreddit3'],
 'science': [],
 'violent': [],
 'health_fitness': [],
 'travel': [],
 'philosophy': [],
 'books_comics': [],
 'personal': [],
 'tech': ['RAZER'],
 'news': [],
 'vehicles': [],
 'music': [],
 'religion': [],
 'video_games': [],
 'food': [],
 'education': [],
 'geography': [],
 'comedy': [],
 'q_and_a': [],
 'hobbies': [],
 'animals': [],
 'stories': ['Hometheater']}

In [56]:
actual = deepcopy(sample)

In [62]:
actual['media'].remove('SouthPark')
actual['media'].remove('NotASubreddit1')
actual['politics'].remove('NotASubreddit2')
actual['politics'].remove('Russian')
actual['politics'].append('russian')
actual['tech'].remove('RAZER')
actual['stories'].remove('Hometheater')
actual['stories'].append('hometheater')
actual['sexual'].append('MommyMilkers')

ValueError: list.remove(x): x not in list

In [65]:
info = {
    'added': ['RAZER','SouthPark','NotASubreddit1','NotASubreddit2'],
    'missing': ['MommyMilkers'],
    'possible': {'Hometheater': ['hometheater'],
                 'Russian': ['russian']}
}

In [60]:
with open('tests/samples/subreddits_grouped_false.json', 'w') as f:
    json.dump(sample, f)

In [66]:
with open('tests/samples/subreddits_grouped_true.json', 'w') as f:
    json.dump(actual, f)

In [67]:
with open('tests/samples/subreddits_grouped_info.json', 'w') as f:
    json.dump(info, f)

In [25]:
groups = files.get_network_paths_grouped()

KeyError: 'Litecoin'